# Preprocessing & DataLoader Visualisation

Walk through every preprocessing step of `ECEiTCNDataset` and inspect the
resulting tensors, labels, and class balance.

In [ ]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from pathlib import Path

from dataset_ecei_tcn import (
    ECEiTCNDataset, create_loaders,
    read_raw_shot, remove_offset, normalize_array, decimate,
)

ROOT = '/global/cfs/cdirs/m5187/proj-share/ECEi_excerpt/dsrpt'
FS   = 1_000_000   # 1 MHz

## 1. Metadata overview

In [ ]:
meta = pd.read_csv(Path(ROOT) / 'meta.csv')
print(f'Columns: {list(meta.columns)}')
print(f'Total shots: {len(meta)}')
print(meta.groupby('split').size())
meta.head(10)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# shot duration distribution
dur_ms = meta['t_disruption']  # already in ms
for sp in meta['split'].unique():
    mask = meta['split'] == sp
    axes[0].hist(dur_ms[mask], bins=30, alpha=0.6, label=sp)
axes[0].set_xlabel('t_disruption (ms)')
axes[0].set_ylabel('Count')
axes[0].set_title('Shot duration distribution')
axes[0].legend()

# shots per split
counts = meta.groupby('split').size()
axes[1].bar(counts.index, counts.values, color=['steelblue', 'firebrick'][:len(counts)])
axes[1].set_ylabel('# Shots')
axes[1].set_title('Shots per split')
for i, v in enumerate(counts.values):
    axes[1].text(i, v + 1, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 2. Raw signal inspection

Load one full shot and show the raw (20, 8) ECEi grid.

In [ ]:
SHOT = meta['shot'].iloc[0]
T_DIS_MS = meta['t_disruption'].iloc[0]
T_DIS = int(T_DIS_MS * 1000)  # in samples

raw = read_raw_shot(ROOT, SHOT)
print(f'Shot {SHOT}: shape = {raw.shape}, t_disruption = {T_DIS_MS:.1f} ms')

In [ ]:
# Plot a few channels across the full shot
time_ms = np.arange(raw.shape[-1]) / (FS / 1000)  # ms
channels = [(0, 0), (10, 4), (19, 7)]  # (row, col) in the 20x8 grid

fig, axes = plt.subplots(len(channels), 1, figsize=(16, 3 * len(channels)), sharex=True)
for ax, (r, c) in zip(axes, channels):
    ax.plot(time_ms, raw[r, c, :], linewidth=0.3, color='k')
    ax.axvline(T_DIS_MS, color='red', linestyle='--', linewidth=1.5, label='t_disruption')
    ax.set_ylabel(f'Ch ({r},{c})')
    ax.legend(loc='upper right', fontsize=9)
    ax.grid(True, alpha=0.2)
axes[-1].set_xlabel('Time (ms)')
axes[0].set_title(f'Raw signal — shot {SHOT}')
plt.tight_layout()
plt.show()

In [ ]:
# Spatial snapshot at a single time instant (middle of the shot)
t_snap = raw.shape[-1] // 2
fig, ax = plt.subplots(figsize=(6, 8))
im = ax.imshow(raw[:, :, t_snap], aspect='auto', cmap='RdBu_r')
ax.set_xlabel('Radial channel')
ax.set_ylabel('Vertical channel')
ax.set_title(f'Spatial snapshot at t = {t_snap/FS*1e3:.1f} ms')
plt.colorbar(im, ax=ax, label='Raw amplitude')
plt.tight_layout()
plt.show()

## 3. Step 1 — DC offset removal

Mean of the first 50 ms (50 000 samples) is subtracted per channel.

In [ ]:
BASELINE_LEN = 50_000
corrected, offset = remove_offset(raw, baseline_length=BASELINE_LEN)

fig, axes = plt.subplots(2, 1, figsize=(16, 6), sharex=True)

r, c = 10, 4
axes[0].plot(time_ms, raw[r, c, :], linewidth=0.3, color='k')
axes[0].axhline(offset[r, c], color='orange', linestyle='--', label=f'Offset = {offset[r,c]:.1f}')
axes[0].axvspan(0, BASELINE_LEN / FS * 1e3, alpha=0.15, color='orange', label='Baseline window')
axes[0].set_title(f'Before offset removal — Ch ({r},{c})')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.2)

axes[1].plot(time_ms, corrected[r, c, :], linewidth=0.3, color='steelblue')
axes[1].axhline(0, color='gray', linestyle='--', alpha=0.5)
axes[1].set_title(f'After offset removal — Ch ({r},{c})')
axes[1].set_xlabel('Time (ms)')
axes[1].grid(True, alpha=0.2)

plt.tight_layout()
plt.show()

In [ ]:
# Offset heatmap across all channels
fig, ax = plt.subplots(figsize=(6, 8))
im = ax.imshow(offset, aspect='auto', cmap='coolwarm')
ax.set_xlabel('Radial channel')
ax.set_ylabel('Vertical channel')
ax.set_title(f'DC offset per channel — shot {SHOT}')
plt.colorbar(im, ax=ax, label='Offset value')
plt.tight_layout()
plt.show()

## 4. Step 2 — Temporal decimation (10×)

Every 10th sample is kept → effective 100 kHz.

In [ ]:
DATA_STEP = 10
decimated = decimate(corrected, DATA_STEP)
time_dec_ms = np.arange(decimated.shape[-1]) / (FS / DATA_STEP / 1000)

print(f'Before decimation: {corrected.shape}  →  After: {decimated.shape}')

# Zoom into a 10-ms window to see the effect
t0, t1 = 500.0, 510.0  # ms
mask_full = (time_ms >= t0) & (time_ms < t1)
mask_dec  = (time_dec_ms >= t0) & (time_dec_ms < t1)

r, c = 10, 4
fig, ax = plt.subplots(figsize=(16, 3))
ax.plot(time_ms[mask_full], corrected[r, c, mask_full],
        linewidth=0.5, color='k', alpha=0.4, label='1 MHz (original)')
ax.plot(time_dec_ms[mask_dec], decimated[r, c, mask_dec],
        linewidth=1.2, color='steelblue', marker='.', markersize=3, label='100 kHz (decimated)')
ax.set_xlabel('Time (ms)')
ax.set_title(f'Decimation comparison — Ch ({r},{c}), {t0}–{t1} ms')
ax.legend()
ax.grid(True, alpha=0.2)
plt.tight_layout()
plt.show()

## 5. Step 3 — Z-score normalisation

Per-channel mean/std computed from training shots; then applied to all splits.

In [ ]:
# Build the dataset (this only reads meta.csv, not the signals)
ds = ECEiTCNDataset(
    root            = ROOT,
    Twarn           = 300_000,
    baseline_length = 50_000,
    data_step       = 10,
    nsub            = 500_000,
    stride          = 500_000,    # non-overlapping
    normalize       = True,
)
ds.summary()

# Compute normalisation from train split
norm_mean, norm_std = ds.compute_norm_stats(split='train', max_shots=40)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 8))

im0 = axes[0].imshow(norm_mean, aspect='auto', cmap='coolwarm')
axes[0].set_title('Per-channel mean (train)')
axes[0].set_xlabel('Radial')
axes[0].set_ylabel('Vertical')
plt.colorbar(im0, ax=axes[0])

im1 = axes[1].imshow(norm_std, aspect='auto', cmap='viridis')
axes[1].set_title('Per-channel std (train)')
axes[1].set_xlabel('Radial')
axes[1].set_ylabel('Vertical')
plt.colorbar(im1, ax=axes[1])

plt.tight_layout()
plt.show()

In [ ]:
# Apply normalisation to the example shot
normalised = normalize_array(decimated, norm_mean, norm_std)

r, c = 10, 4
fig, axes = plt.subplots(2, 2, figsize=(16, 7))

# Time series comparison
axes[0, 0].plot(time_dec_ms, decimated[r, c, :], linewidth=0.3, color='k')
axes[0, 0].set_title(f'Before normalisation — Ch ({r},{c})')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].grid(True, alpha=0.2)

axes[0, 1].plot(time_dec_ms, normalised[r, c, :], linewidth=0.3, color='steelblue')
axes[0, 1].set_title(f'After normalisation — Ch ({r},{c})')
axes[0, 1].set_ylabel('z-score')
axes[0, 1].grid(True, alpha=0.2)

# Histograms
axes[1, 0].hist(decimated[r, c, :], bins=100, color='gray', alpha=0.7)
axes[1, 0].set_xlabel('Amplitude')
axes[1, 0].set_title('Value distribution (before)')

axes[1, 1].hist(normalised[r, c, :], bins=100, color='steelblue', alpha=0.7)
axes[1, 1].set_xlabel('z-score')
axes[1, 1].set_title('Value distribution (after)')

plt.tight_layout()
plt.show()

In [ ]:
# Distribution across ALL channels before vs. after normalisation
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Flatten all channels, subsample time for speed
step = 100
vals_before = decimated[:, :, ::step].flatten()
vals_after  = normalised[:, :, ::step].flatten()

axes[0].hist(vals_before, bins=200, color='gray', alpha=0.7)
axes[0].set_title('All-channel amplitude (before norm)')
axes[0].set_xlabel('Amplitude')
axes[0].set_xlim(np.percentile(vals_before, [0.5, 99.5]))

axes[1].hist(vals_after, bins=200, color='steelblue', alpha=0.7)
axes[1].set_title('All-channel z-score (after norm)')
axes[1].set_xlabel('z-score')
axes[1].set_xlim(np.percentile(vals_after, [0.5, 99.5]))

plt.tight_layout()
plt.show()

## 6. Per-timestep labels & weights

The label is 0 (clear) until `Twarn` before disruption, then 1 (disruptive).

In [ ]:
# Show the label on the full shot time axis
Twarn_ms = 300  # ms

fig, axes = plt.subplots(2, 1, figsize=(16, 5), sharex=True)

r, c = 10, 4
axes[0].plot(time_dec_ms, normalised[r, c, :], linewidth=0.3, color='k')
axes[0].set_ylabel('z-score')
axes[0].set_title(f'Normalised signal — Ch ({r},{c}), shot {SHOT}')
axes[0].axvline(T_DIS_MS, color='red', ls='--', lw=1.5, label='t_disruption')
axes[0].axvline(T_DIS_MS - Twarn_ms, color='orange', ls='--', lw=1.5, label=f't_dis − Twarn ({Twarn_ms} ms)')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.2)

# Per-timestep label
label_full = np.zeros(len(time_dec_ms), dtype=np.float32)
d_idx_dec = int((T_DIS - 300_000) / DATA_STEP)
d_idx_dec = max(0, min(d_idx_dec, len(label_full)))
label_full[d_idx_dec:] = 1.0

axes[1].fill_between(time_dec_ms, 0, label_full, color='firebrick', alpha=0.4, label='Disruptive (1)')
axes[1].fill_between(time_dec_ms, 0, 1 - label_full, color='steelblue', alpha=0.2, label='Clear (0)')
axes[1].set_ylabel('Label')
axes[1].set_xlabel('Time (ms)')
axes[1].set_title('Per-timestep binary label (Twarn = 300 ms)')
axes[1].set_ylim(-0.05, 1.15)
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.2)

plt.tight_layout()
plt.show()

## 7. Subsequence tiling

Each shot is split into fixed-length windows (500 ms). Windows that straddle the
disruption boundary have a label transition inside.

In [ ]:
# Find all subsequences for this shot
shot_meta_idx = np.where(ds.shots == SHOT)[0][0]
seq_mask = ds.seq_shot_idx == shot_meta_idx
seq_starts = ds.seq_start[seq_mask]
seq_stops  = ds.seq_stop[seq_mask]
seq_disrupt = ds.seq_disrupt_local[seq_mask]

print(f'Shot {SHOT}: {seq_mask.sum()} subsequences')

fig, ax = plt.subplots(figsize=(16, 3))
for i, (a, b, d) in enumerate(zip(seq_starts, seq_stops, seq_disrupt)):
    color = 'firebrick' if d >= 0 else 'steelblue'
    ax.barh(0, (b - a) / FS * 1e3, left=a / FS * 1e3, height=0.6,
            color=color, alpha=0.5, edgecolor='k', linewidth=0.5)
ax.axvline(T_DIS_MS, color='red', ls='--', lw=2, label='t_disruption')
ax.axvline(T_DIS_MS - Twarn_ms, color='orange', ls='--', lw=1.5, label='label boundary')
ax.set_xlabel('Time (ms)')
ax.set_title(f'Subsequence windows for shot {SHOT} (red = contains disruption label)')
ax.set_yticks([])
ax.legend(loc='upper left')
plt.tight_layout()
plt.show()

## 8. DataLoader end-to-end test

Pull a batch from the train loader and verify shapes + label distribution.

In [ ]:
loaders = create_loaders(ds, batch_size=4, num_workers=0)
print('Splits available:', list(loaders.keys()))

split_name = 'train' if 'train' in loaders else list(loaders.keys())[0]
batch = next(iter(loaders[split_name]))
X_b, target_b, weight_b = batch

print(f'\nBatch from "{split_name}":')
print(f'  X      : {X_b.shape}  dtype={X_b.dtype}')
print(f'  target : {target_b.shape}  dtype={target_b.dtype}')
print(f'  weight : {weight_b.shape}  dtype={weight_b.dtype}')
print(f'  label frac per sample: {[f"{t.mean():.2f}" for t in target_b]}')

In [ ]:
# Visualise one sample from the batch
idx = 0
x_np = X_b[idx].numpy()          # (20, 8, T_sub)
t_np = target_b[idx].numpy()     # (T_sub,)
w_np = weight_b[idx].numpy()     # (T_sub,)
T_sub = x_np.shape[-1]
t_ax = np.arange(T_sub) / (FS / ds.data_step / 1000)  # ms

fig, axes = plt.subplots(4, 1, figsize=(16, 10), sharex=True,
                          gridspec_kw={'height_ratios': [3, 3, 1, 1]})

# Signal (one channel)
axes[0].plot(t_ax, x_np[10, 4, :], linewidth=0.4, color='k')
axes[0].set_ylabel('z-score')
axes[0].set_title('Preprocessed signal — Ch (10, 4)')
axes[0].grid(True, alpha=0.2)

# All 160 channels as heatmap
flat = x_np.reshape(160, -1)
axes[1].imshow(flat, aspect='auto', cmap='RdBu_r',
               vmin=np.percentile(flat, 1), vmax=np.percentile(flat, 99),
               extent=[t_ax[0], t_ax[-1], 159, 0])
axes[1].set_ylabel('Channel')
axes[1].set_title('All 160 channels (20×8 flattened)')

# Per-timestep label
axes[2].fill_between(t_ax, 0, t_np, color='firebrick', alpha=0.5)
axes[2].set_ylabel('Label')
axes[2].set_ylim(-0.05, 1.15)

# Weight
axes[3].plot(t_ax, w_np, color='darkorange', linewidth=1)
axes[3].set_ylabel('Weight')
axes[3].set_xlabel('Time (ms) within subsequence')

plt.tight_layout()
plt.show()

## 9. Class balance statistics

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

for i, sp in enumerate(np.unique(ds.splits)):
    idx = ds.get_split_indices(sp)
    n_dis  = int(ds.seq_has_disrupt[idx].sum())
    n_clr  = len(idx) - n_dis

    axes[0].bar(i * 3, n_clr, color='steelblue', width=0.8, label='Clear' if i == 0 else '')
    axes[0].bar(i * 3 + 1, n_dis, color='firebrick', width=0.8, label='Disruptive' if i == 0 else '')
    axes[0].text(i * 3, n_clr + 1, str(n_clr), ha='center', fontsize=9)
    axes[0].text(i * 3 + 1, n_dis + 1, str(n_dis), ha='center', fontsize=9)

axes[0].set_xticks([i * 3 + 0.5 for i in range(len(np.unique(ds.splits)))])
axes[0].set_xticklabels(np.unique(ds.splits))
axes[0].set_ylabel('# Subsequences')
axes[0].set_title('Subsequences per split (clear vs disruptive)')
axes[0].legend()

# Fraction of disruptive time steps per subsequence
T_sub = ds.nsub // ds.data_step
fracs = []
for dl in ds.seq_disrupt_local:
    if dl < 0:
        fracs.append(0.0)
    else:
        d = min(dl // ds.data_step, T_sub)
        fracs.append((T_sub - d) / T_sub)
fracs = np.array(fracs)

axes[1].hist(fracs, bins=50, color='gray', alpha=0.7, edgecolor='k')
axes[1].set_xlabel('Fraction of disruptive time steps')
axes[1].set_ylabel('# Subsequences')
axes[1].set_title('Distribution of disruptive fraction per subsequence')

plt.tight_layout()
plt.show()

print(f'\nClass weights: pos_weight = {ds.pos_weight:.3f}, neg_weight = {ds.neg_weight:.3f}')